In [3]:
import numpy as np
import create_grids, create_polynomials, compute_MC_Residual_Histogram, compute_MC_Residual_Poly

The calibration portion of the replication exercise heavily follows Winberry 2020

First we set the model parameters:

In [4]:
# from calibrated version of Winberry, 2020

# Technology 
θ = .21 # capital coefficient
ν = .64 # labor coefficient
δ = .025 # depreciation (annual rate)
ρ = .9 # persistence of idiosyncratic income shocks on productivity
a̅ = 0 #no fixed cost region upper bound
a̲ = -a̅  # no fixed cost region lower bound
# Preferences
β = .99 # discount rate
σ = 1 #curvature on utility func
ϕ = 1/2 #adjustment cost for capital
N_SS = 1/3  # hours worked in SS
χ = 1 #disutility of labor supply 
η_0 = 0 #weight on past habit
# Tax code
τ = .35  # linear tax
δ̂=  .119 # tax depreciation rate
PV = (δ̂/(1-β)*(1-δ̂)) # present value of depreciation allowances

# Calibration parameters
Params = [0.053187, 0.703137,2.95000, 0.745793]
σ_prod = Params[0]
ψ_cap =  Params[1]
ψ_cap_2 = Params[2]
η_1 = Params[3]


Next we set the parameters that control the approximations (chebyshev and distribution):

In [3]:
# From Winberry 2020 replcation
k_rep_SS = (θ*N_SS**(ν))/(1/β-(1-δ))**(1/(1-θ))
w_rep_SS = ν*N_SS**(ν-1)*k_rep_SS**(θ)

# Order of approx of the value function (chebyshev series cutoff) 
approx_n_prod= 3 # order of CE chebyshev poly. approx in productivity
approx_n_cap = 5 # order of CE chebyshev poly. approx in capital
n_states = approx_n_prod*approx_n_cap # total order of CE chebyshev poly. approx
# Grid bounds
prod_min = -3*σ_prod/np.sqrt(1-ρ**2) #(-3 s.d. away)
prod_max = 3*σ_prod/np.sqrt(1-ρ**2) #(+3 s.d. awy)
capital_min = np.max((0, 0.025+ (2/(1+20*ψ_cap_2))*np.exp(-(1/ψ_cap_2))*(np.exp(prod_min))**(1/(1-θ))*k_rep_SS)) #minimum capital achievable in SS under the worst case parametization
capital_max = np.max((capital_min, 2.5- (40/(1.0+20*ψ_cap_2))*np.exp(-(1/ψ_cap_2))*(np.exp(prod_max))**(1/(1-θ))*k_rep_SS)) #maximum capital achievable in SS under parameterization

# Shocks
n_shocks=3

#Finer grid for analyzing policy funcs and histogram properties
approx_n_prod_fine = 60
approx_n_cap_fine = 150
n_states_fine = approx_n_prod_fine*approx_n_cap_fine 

# Iterations for value function updating
max_iter = 100
tol = 1e-6
howard_iter = 500 # number of iterations in Howard PI step at SS
w = 0.5 # weight on old iterations

Finally we  calibrate the approximation of the underlying distribution:

In [4]:
# Approximation of distribution heavily adapted from Winberry, 2020
order_n = 2 # order of the polynomial for the exponential family approx
n_prod_quad = 10 # number of quadrature points to use for productivity
n_cap_quad = 25 # number of quadrature points to use for capital
n_state_quad = n_prod_quad*n_cap_quad # number of quadrature points overall for nd integral
n_measure_coeff = order_n*(order_n+1)/2+ order_n # total num of coefficients needed to compute for exp family


## Finding the Steady State

We begin solving for the steady state of the file. First we construct the grids we need (there are many) using the `create_grids` function.

In [ ]:
create_grids()